In [1]:
import os
import glob
import open3d
import numpy as np

from PIL import Image
from copy import deepcopy
from utils.depth_camera import DepthCamera

In [2]:
def read_first_depth_img(experiment_dir, device_id):
    """
    Read the first depth image from a trial.
    """
    depth_img_path = glob.glob(f"{experiment_dir}/trial_0/global/device-{device_id}/*.depth.png")[0]
    device = DepthCamera(f"device-{device_id}", f"{experiment_dir}/metadata/device-{device_id}-aligned.json")
    return device.depth_to_point_cloud(depth_img_path)


def read_extrinsics(experiment_dir, device_id):
    """
    Read the extrinsics from a trial.
    """
    return np.loadtxt(f"{experiment_dir}/trial_1/global/transformations/device-{device_id}.txt")


def write_extrinsics(experiment_dir, trial, device_id, transformation):
    """
    write the extrinsics from a trial.
    """
    if not os.path.exists(f"{experiment_dir}/{trial}/global/transformations"):
        os.makedirs(f"{experiment_dir}/{trial}/global/transformations")
        
    return np.savetxt(f"{experiment_dir}/{trial}/global/transformations/device-{device_id}.txt", transformation)


def preprocess_pcd(pcd, voxel_size, down_sample=True):
    if down_sample:
        pcd = open3d.voxel_down_sample(pcd, voxel_size)

    radius_normal = voxel_size * 2
    open3d.geometry.estimate_normals(pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    pcd_fpfh = open3d.registration.compute_fpfh_feature(pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd, pcd_fpfh


def execute_global_registration(source_down, target_down, source_feat, target_feat, n_ransac, threshold):
    result = open3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_feat, target_feat, threshold,
        open3d.registration.TransformationEstimationPointToPoint(False), n_ransac, 
        [open3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), open3d.registration.CorrespondenceCheckerBasedOnDistance(threshold)],
        open3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result


def refine_registration(source, target, distance_threshold, trans_init):
    result = open3d.registration.registration_icp(
        source, target, distance_threshold, trans_init,
        open3d.registration.TransformationEstimationPointToPlane(),
        open3d.registration.ICPConvergenceCriteria(max_iteration=1000)
    )
    return result


def visualize(source, target, transformation):
    source_temp = deepcopy(source)
    target_temp = deepcopy(target)
    
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    
    source_temp.transform(transformation)
    
    open3d.visualization.draw_geometries([source_temp, target_temp])
    
    
def rotate_transformation_matrix(t, rx, ry, rz):
    # Convert degrees to radians
    rx, ry, rz = np.radians(rx), np.radians(ry), np.radians(rz)

    RX = np.array([
        [1, 0, 0, 0],
        [0, np.cos(rx), -np.sin(rx), 0],
        [0, np.sin(rx), np.cos(rx), 0],
        [0, 0, 0, 1]
    ])

    RY = np.array([
        [np.cos(ry), 0, np.sin(ry), 0],
        [0, 1, 0, 0],
        [-np.sin(ry), 0, np.cos(ry), 0],
        [0, 0, 0, 1]
    ])

    RZ = np.array([
        [np.cos(rz), -np.sin(rz), 0, 0],
        [np.sin(rz), np.cos(rz), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    return np.dot(np.dot(np.dot(t, RZ), RY), RX)

In [3]:
dev_0_pcd = read_first_depth_img("data/raw_data/exp_9", 0)
dev_1_pcd = read_first_depth_img("data/raw_data/exp_9", 1)
dev_2_pcd = read_first_depth_img("data/raw_data/exp_9", 2)

In [8]:
open3d.io.write_point_cloud("temp/dev_0.pcd", dev_0_pcd)
open3d.io.write_point_cloud("temp/dev_1.pcd", dev_1_pcd)
open3d.io.write_point_cloud("temp/dev_2.pcd", dev_2_pcd)

True

In [5]:
dev_0_trans_init = np.loadtxt("temp/dev_0.txt")
dev_1_trans_init = np.loadtxt("temp/dev_1.txt")
dev_2_trans_init = np.loadtxt("temp/dev_2.txt")

In [6]:
dev_0_pcd = open3d.voxel_down_sample(dev_0_pcd, 0.03)
dev_1_pcd = open3d.voxel_down_sample(dev_1_pcd, 0.03)
dev_2_pcd = open3d.voxel_down_sample(dev_2_pcd, 0.03)

open3d.geometry.estimate_normals(dev_0_pcd)
open3d.geometry.estimate_normals(dev_1_pcd)
open3d.geometry.estimate_normals(dev_2_pcd)

True

In [12]:
dev_0_trans_init = read_extrinsics("data/raw_data/exp_5", 0)
dev_1_trans_init = read_extrinsics("data/raw_data/exp_5", 1)
dev_2_trans_init = read_extrinsics("data/raw_data/exp_5", 2)

In [6]:
dev_0_pcd.transform(dev_0_trans_init)
dev_1_pcd.transform(dev_1_trans_init)
dev_2_pcd.transform(dev_2_trans_init)

geometry::PointCloud with 123815 points.

In [7]:
open3d.visualization.draw_geometries([dev_0_pcd, dev_1_pcd, dev_2_pcd])

In [15]:
global_merged_pcd = open3d.geometry.PointCloud()

global_merged_pcd += dev_0_pcd
global_merged_pcd += dev_1_pcd
global_merged_pcd += dev_2_pcd

global_merged_pcd = open3d.voxel_down_sample(global_merged_pcd, 0.03)

open3d.visualization.draw_geometries([global_merged_pcd])

# open3d.io.write_point_cloud("temp/larc/global_merged.pcd", global_merged_pcd)

# open3d.io.write_point_cloud("data/reference/larc_kitchen_3cams.pcd", global_merged_pcd)

In [18]:
global_pcd = open3d.io.read_point_cloud("temp/lidar_1637299401488642900.pcd")
global_pcd = open3d.voxel_down_sample(global_pcd, 0.03)
global_pcd.paint_uniform_color([1, 0.706, 0])

open3d.geometry.estimate_normals(global_pcd)

True

In [19]:
open3d.visualization.draw_geometries([dev_0_pcd, dev_1_pcd, dev_2_pcd, global_pcd])

In [15]:

T = np.identity(4)

T = rotate_transformation_matrix(T, -30, 0, 180)

T[:, 3] = np.array([-1.5, 2.8, -2.5, 1])

result = open3d.registration.registration_icp(
    dev_0_pcd, global_pcd, 0.05, T,
    open3d.registration.TransformationEstimationPointToPlane(),
    open3d.registration.ICPConvergenceCriteria(max_iteration=2000)
)

print(result)

visualize(dev_0_pcd, global_pcd, result.transformation)

dev_0_trans_init = deepcopy(result.transformation)

registration::RegistrationResult with fitness = 0.777096, inlier_rmse = 0.024847, and correspondence_set size of 16462
Access transformation to get result.


In [16]:
T = np.identity(4)

T = rotate_transformation_matrix(T, -30, 180, 180)

T[:, 3] = np.array([-1.9, 2.8, 4.2, 1])

result = open3d.registration.registration_icp(
    dev_1_pcd, global_pcd, 0.03, T,
    open3d.registration.TransformationEstimationPointToPlane(),
    open3d.registration.ICPConvergenceCriteria(max_iteration=2000)
)

print(result)

visualize(dev_1_pcd, global_pcd, T)
visualize(dev_1_pcd, global_pcd, result.transformation)

dev_1_trans_init = deepcopy(result.transformation)

registration::RegistrationResult with fitness = 0.641067, inlier_rmse = 0.018456, and correspondence_set size of 12379
Access transformation to get result.


In [37]:
T = np.identity(4)

T = rotate_transformation_matrix(T, -30, 85, 180)

T[:, 3] = np.array([2.8, 3.2, 0.5, 1])

result = open3d.registration.registration_icp(
    dev_2_pcd, global_pcd, 0.03, T,
    open3d.registration.TransformationEstimationPointToPlane(),
    open3d.registration.ICPConvergenceCriteria(max_iteration=2000)
)

print(result)

visualize(dev_2_pcd, global_pcd, T)
visualize(dev_2_pcd, global_pcd, result.transformation)

dev_2_trans_init = deepcopy(result.transformation)

registration::RegistrationResult with fitness = 0.648896, inlier_rmse = 0.019138, and correspondence_set size of 17016
Access transformation to get result.


In [6]:
global_pcd = open3d.io.read_point_cloud("temp/lidar_1637299401488642900.pcd")
global_pcd_down, global_pcd_fpfh = preprocess_pcd(global_pcd, voxel_size=0.03)

dev_0_pcd_down, dev_0_pcd_fpfh = preprocess_pcd(dev_0_pcd, 0.03)
dev_1_pcd_down, dev_1_pcd_fpfh = preprocess_pcd(dev_1_pcd, 0.03)
dev_2_pcd_down, dev_2_pcd_fpfh = preprocess_pcd(dev_2_pcd, 0.03)

In [7]:
icp_result = refine_registration(dev_0_pcd_down, global_pcd_down, 0.05, np.identity(4))
print(icp_result)

registration::RegistrationResult with fitness = 0.083868, inlier_rmse = 0.029257, and correspondence_set size of 1737
Access transformation to get result.


In [8]:
visualize(dev_0_pcd_down, global_pcd_down, icp_result.transformation)

In [131]:
dev_1_pcd_down.transform(icp_result.transformation)

geometry::PointCloud with 29618 points.

In [132]:
target = dev_1_pcd_down + dev_2_pcd_down
icp_result = refine_registration(dev_0_pcd_down, target, 0.05, np.identity(4))
print(icp_result)

dev_0_trans_init = np.dot(icp_result.transformation, dev_0_trans_init)

registration::RegistrationResult with fitness = 0.409074, inlier_rmse = 0.024551, and correspondence_set size of 12037
Access transformation to get result.


In [134]:
visualize(dev_0_pcd_down, target, icp_result.transformation)

In [133]:
dev_0_pcd_down.transform(icp_result.transformation)

geometry::PointCloud with 29425 points.

In [135]:
source = dev_0_pcd_down + dev_1_pcd_down + dev_2_pcd_down
open3d.visualization.draw_geometries([source])

In [136]:
icp_result = refine_registration(source, global_pcd_down, 0.05, np.identity(4))
print(icp_result)

dev_0_trans_init = np.dot(icp_result.transformation, dev_0_trans_init)
dev_1_trans_init = np.dot(icp_result.transformation, dev_1_trans_init)
dev_2_trans_init = np.dot(icp_result.transformation, dev_2_trans_init)

registration::RegistrationResult with fitness = 0.594485, inlier_rmse = 0.026896, and correspondence_set size of 52773
Access transformation to get result.


In [137]:
visualize(source, global_pcd_down, icp_result.transformation)

In [9]:
dev_0_trans_init = np.loadtxt("temp/larc/dev_0.txt")
dev_1_trans_init = np.loadtxt("temp/larc/dev_1.txt")
dev_2_trans_init = np.loadtxt("temp/larc/dev_2.txt")

In [8]:
for trial in os.listdir("data/raw_data/exp_9/"):
    if trial.startswith("trial"):
        write_extrinsics("data/raw_data/exp_9", trial, 0, dev_0_trans_init)
        write_extrinsics("data/raw_data/exp_9", trial, 1, dev_1_trans_init)
        write_extrinsics("data/raw_data/exp_9", trial, 2, dev_2_trans_init)

In [15]:
ransac_result = execute_global_registration(dev_1_pcd_down, dev_2_pcd_down, dev_1_pcd_fpfh, dev_2_pcd_fpfh, 4, 0.05)
# icp_result = refine_registration(dev_0_pcd_down, dev_1_pcd_down, 0.05, ransac_result.transformation)

In [16]:
ransac_result

registration::RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.

In [9]:
dev_0_pcd_down.transform(icp_result.transformation)

geometry::PointCloud with 37129 points.

In [10]:
open3d.visualization.draw_geometries([dev_0_pcd_down, dev_1_pcd_down])

In [5]:
camera = DepthCamera("device-3", "metadata/device-3-aligned.json")

conv_pcd = camera.depth_to_point_cloud("temp/frame-1658916263265080400.depth.png")

org_pcd = open3d.io.read_point_cloud("temp/frame-1658916263265080400.pcd")

conv_pcd.paint_uniform_color([1, 0, 0])
org_pcd.paint_uniform_color([0, 1, 0])

T = np.identity(4)
T[0, 3] = 1

# org_pcd.transform(T)

open3d.visualization.draw_geometries([conv_pcd, org_pcd])

In [6]:
def convert_to_point_cloud(intrinsic, depth_image, depth_scale=1000, step=1):
    fx, fy = intrinsic.get_focal_length()
    cx, cy = intrinsic.get_principal_point()
    
    if depth_image.mode != "I":
        raise Exception("Depth image is not in intensity format")


    points = []
    for v in range(0, depth_image.height, step):
        for u in range(0, depth_image.width, step):
            z = depth_image.getpixel((u,v)) / depth_scale
            if z == 0: 
                continue
            x = (u - cx) * z / fx
            y = (v - cy) * z / fy
            points.append([x, y, z])
            
    xpcd = open3d.geometry.PointCloud()
    xpcd.points = open3d.utility.Vector3dVector(points)
    
    return xpcd

def read_intrinsic(filepath, width=640, height=480):
    m = np.loadtxt(filepath, dtype=np.float32)
    intrinsic = open3d.camera.PinholeCameraIntrinsic(width, height, m[0, 0], m[1, 1], m[0, 2], m[1, 2])
    return intrinsic

In [8]:
intrinsic = read_intrinsic("D:/Projects/Research/LARCDataset/v1/larc-kitchen/camera-intrinsics.txt")
depth_scale = 4000

In [12]:
conv_pcd = convert_to_point_cloud(intrinsic, Image.open("temp/frame-1658916263265080400.depth.png").convert("I"), depth_scale)

org_pcd = open3d.io.read_point_cloud("temp/frame-1658916263265080400.pcd")

conv_pcd.paint_uniform_color([1, 0, 0])
org_pcd.paint_uniform_color([0, 1, 0])

T = np.identity(4)
T[0, 3] = 1

# org_pcd.transform(T)

open3d.visualization.draw_geometries([conv_pcd, org_pcd])

In [44]:
dev_0_pcd = open3d.io.read_point_cloud("temp/dev_0.pcd")
dev_1_pcd = open3d.io.read_point_cloud("temp/dev_1.pcd")
dev_2_pcd = open3d.io.read_point_cloud("temp/dev_2.pcd")

In [5]:
dev_0_pcd.transform(np.loadtxt("temp/larc/dev_0.txt"))
dev_1_pcd.transform(np.loadtxt("temp/larc/dev_1.txt"))
dev_2_pcd.transform(np.loadtxt("temp/larc/dev_2.txt"))

geometry::PointCloud with 225478 points.

In [6]:
open3d.visualization.draw_geometries([dev_1_pcd, dev_2_pcd, dev_0_pcd])

In [32]:
global_pcd = dev_1_pcd + dev_2_pcd

global_pcd_down, global_pcd_fpfh = preprocess_pcd(global_pcd, 0.03)
dev_0_pcd_down, dev_0_pcd_fpfh = preprocess_pcd(dev_0_pcd, 0.03)

In [37]:
ransac_result = execute_global_registration(dev_0_pcd_down, global_pcd_down, dev_0_pcd_fpfh, global_pcd_fpfh, 4, 0.05)
icp_result = refine_registration(dev_0_pcd_down, global_pcd_down, 0.05, ransac_result.transformation)

print(ransac_result)
print(icp_result)

registration::RegistrationResult with fitness = 0.000000, inlier_rmse = 0.000000, and correspondence_set size of 0
Access transformation to get result.
registration::RegistrationResult with fitness = 0.030222, inlier_rmse = 0.031064, and correspondence_set size of 903
Access transformation to get result.


In [38]:
dev_0_pcd_down.transform(icp_result.transformation)

geometry::PointCloud with 29879 points.

In [39]:
open3d.visualization.draw_geometries([dev_0_pcd_down, global_pcd_down])

In [82]:
global_pcd = open3d.io.read_point_cloud("temp/lidar_1637299401488642900.pcd")
dev_0_pcd = open3d.io.read_point_cloud("temp/dev_0.pcd")

global_pcd.paint_uniform_color([1, 0.706, 0])
dev_0_pcd.paint_uniform_color([0, 0.651, 0.929])

# dev_0_pcd.transform(np.loadtxt("temp/dev_0.txt"))

# T = np.identity(4)
T = np.loadtxt("temp/dev_0.txt")

T = rotate_transformation_matrix(T, 0, -25, 0)

# T[:, 3] = np.array([-4, 1, 3, 1])

# dev_0_pcd.transform(T)

result = open3d.registration.registration_icp(
    dev_1_pcd, global_pcd, 0.03, T,
    open3d.registration.TransformationEstimationPointToPlane(False),
    open3d.registration.ICPConvergenceCriteria(max_iteration=5000)
)

print(result)

dev_0_pcd.transform(result.transformation)

open3d.visualization.draw_geometries([global_pcd, dev_0_pcd])

registration::RegistrationResult with fitness = 0.023173, inlier_rmse = 0.019648, and correspondence_set size of 4012
Access transformation to get result.


In [ ]:
np.savetxt("temp/dev_1.txt", result.transformation)

In [31]:
global_pcd = open3d.io.read_point_cloud("temp/lidar_1637299401488642900.pcd")

open3d.geometry.estimate_normals(global_pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=0.03 * 2, max_nn=30))

True

In [32]:
open3d.geometry.estimate_normals(dev_0_pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=0.03 * 2, max_nn=30))
open3d.geometry.estimate_normals(dev_1_pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=0.03 * 2, max_nn=30))
open3d.geometry.estimate_normals(dev_2_pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=0.03 * 2, max_nn=30))

True

In [34]:
global_pcd.paint_uniform_color([1, 0.706, 0])
dev_1_pcd.paint_uniform_color([0, 0.651, 0.929])

result = open3d.registration.registration_icp(
    dev_1_pcd, global_pcd, 0.05, dev_1_trans_init,
    open3d.registration.TransformationEstimationPointToPlane(),
    open3d.registration.ICPConvergenceCriteria(max_iteration=1000)
)

print(result)

dev_1_pcd.transform(result.transformation)

open3d.visualization.draw_geometries([global_pcd, dev_1_pcd])



registration::RegistrationResult with fitness = 0.168534, inlier_rmse = 0.029552, and correspondence_set size of 29703
Access transformation to get result.


In [87]:
dev_1_pcd = open3d.io.read_point_cloud("temp/dev_1.pcd")
dev_0_pcd = open3d.io.read_point_cloud("temp/dev_0.pcd")

dev_1_pcd.paint_uniform_color([1, 0.706, 0])
dev_0_pcd.paint_uniform_color([0, 0.651, 0.929])

dev_1_pcd.transform(np.loadtxt("temp/dev_1.txt"))
# dev_0_pcd.transform(np.loadtxt("temp/dev_0.txt"))

# T = np.identity(4)
T = np.loadtxt("temp/dev_0.txt")

T = rotate_transformation_matrix(T, 0, -25, 0)

# T[:, 3] = np.array([-4, 1, 3, 1])

# dev_0_pcd.transform(T)

result = open3d.registration.registration_icp(
    dev_0_pcd, dev_1_pcd, 0.05, T,
    open3d.registration.TransformationEstimationPointToPoint(False),
    open3d.registration.ICPConvergenceCriteria(max_iteration=1000)
)

print(result)

dev_0_pcd.transform(result.transformation)

open3d.visualization.draw_geometries([dev_1_pcd, dev_0_pcd])

registration::RegistrationResult with fitness = 0.272207, inlier_rmse = 0.022144, and correspondence_set size of 52082
Access transformation to get result.


In [88]:
np.savetxt("temp/dev_0.txt", result.transformation)

In [2]:
pcd = open3d.io.read_point_cloud("data/point_clouds/exp_5/trial_1/subject-1/01/1663141115629134100.global.pcd")

In [3]:
open3d.visualization.draw_geometries([pcd])

In [17]:
global_pcd_files = glob.glob(os.path.join("data/point_clouds/exp_5", "trial_*\subject-*\*\*.global.pcd"))

In [18]:
for f in global_pcd_files:
    os.remove(f)